In [1]:
# Creating a merchant table 
import pandas as pd
import math
import random

In [2]:
# Creating our merchants details
merchant_num = 283726
merchant_name = ['UnityTrust Bank Lagos, Nigeria', 'Savannah Capital Bank Nairobi, Kenya',
                 'GoldCoast Heritage Bank Accra, Ghana', 'Ubuntu Commercial Bank Johannesburg, South Africa',
                 'Nile Delta Bank Cairo, Egypt', 'BlueRiver Financial Group London, United Kingdom',
                 'Alpine Trust Bank Zurich, Switzerland', 'Danube Union Bank Vienna, Austria',
                 'Nordic Growth Bank Stockholm, Sweden', 'Mediterraneo Bank Rome, Italy']
merchant_location = ['Lagos, Nigeria', 'Nairobi, Kenya', 'Accra, Ghana',
                     'Johannesburg, South Africa', 'Cairo, Egypt', 'London, United Kingdom',
                     'Zurich, Switzerland', 'Vienna, Austria', 'Stockholm, Sweden', 'Rome, Italy']

In [3]:
# Calculate distribution
per_merchant = merchant_num // len(merchant_name) # base count
remainder = merchant_num % len(merchant_name) # leftover

# Build aligned dataset
data = []
for i, (m_name, m_loc) in enumerate(zip(merchant_name, merchant_location)):
    count = per_merchant + (1 if i < remainder else 0)  
    data.extend([(m_name, m_loc)] * count)

# Shuffle for realism
random.shuffle(data)

# Assign sequential IDs after shuffle
df = pd.DataFrame(data, columns=["merchant_name", "merchant_location"])
df.insert(0, "id", range(1, len(df) + 1))

# Save to CSV
df.to_csv("merchants.csv", index=False)

print("✅ merchants.csv created with", len(df), "rows")
print(df.head(15))

✅ merchants.csv created with 283726 rows
    id                                      merchant_name  \
0    1                       Nile Delta Bank Cairo, Egypt   
1    2   BlueRiver Financial Group London, United Kingdom   
2    3  Ubuntu Commercial Bank Johannesburg, South Africa   
3    4  Ubuntu Commercial Bank Johannesburg, South Africa   
4    5               GoldCoast Heritage Bank Accra, Ghana   
5    6   BlueRiver Financial Group London, United Kingdom   
6    7                  Danube Union Bank Vienna, Austria   
7    8                       Nile Delta Bank Cairo, Egypt   
8    9               GoldCoast Heritage Bank Accra, Ghana   
9   10                     UnityTrust Bank Lagos, Nigeria   
10  11                      Mediterraneo Bank Rome, Italy   
11  12                  Danube Union Bank Vienna, Austria   
12  13  Ubuntu Commercial Bank Johannesburg, South Africa   
13  14               GoldCoast Heritage Bank Accra, Ghana   
14  15  Ubuntu Commercial Bank Johannesburg,

In [4]:
import psycopg2

In [5]:
df.dtypes

id                    int64
merchant_name        object
merchant_location    object
dtype: object

### Loading my cleaned dataset to postgresSQL

In [7]:
def extract():
    df = pd.read_csv("merchants.csv")
    return df

def load(df):
    try:
        connection = psycopg2.connect(
                host = "localhost",
                dbname = "Fraud_Alert",
                user = "postgres",
                password = "BBK#0027"
        )

        con = connection.cursor()

        for _, row in df.iterrows():
            con.execute("""
                        INSERT INTO merchants(id, merchant_name, merchant_location)
                        VALUES (%s,%s,%s) """, tuple (row)
                        )
            
        connection.commit()
        con.close()
        connection.close()
        print("Data loaded successfully into merchants table.")

    except Exception as e:
        print("Error", e)

if __name__ == "__main__":
    raw_data = extract()
    load(raw_data)

Data loaded successfully into merchants table.
